# Парсинг

In [ ]:
!pip install spacy_conll

In [ ]:
!python -m spacy download ru_core_news_sm

In [9]:
import re
import json
import pandas as pd

In [31]:
with open('corpus.json') as json_file:
    dictionary = json.load(json_file)

In [34]:
from spacy_conll import init_parser
from spacy.symbols import ORTH
import spacy

nlp = init_parser("ru_core_news_sm", 'spacy', include_headers=True)

In [39]:
specials = ['мин.', 'ст. л.', 'ч. л.', '180°C']

In [44]:
link = 'https://www.gastronom.ru/recipe/'
for i in dictionary:
  current_link = f'{link}{i}'
  doc = nlp(dictionary[i][0])
  for spec in specials:
    if spec[:2] in doc.text:
      if spec[:-2] in doc.text:
        nlp.tokenizer.add_special_case(spec, [{ORTH: spec}])
  conll = doc._.conll_str
  conll = re.sub('([0-9]+)\t{5}SPACE\tSPACE.+\n', '', conll)
  conll = re.sub('\n\n', f'\n\n# source = {current_link}\n# name = {dictionary[i][1]}\n', conll)
  with open('corpus_parsed.conllu', 'a', encoding='utf-8') as f:
    f.write(f'\n# source = {current_link}\n# name = {dictionary[i][1]}\n')
    f.write(conll)